In [1]:
import os
import re
import matplotlib.pyplot as plt

In [2]:
def extract_values(file_path, structured=True):
    obj_acc_1 = None
    rel_acc_1 = None
    triplet_acc_50 = None
    total_params = None

    with open(file_path, 'r') as file:
        for line in file:
            if "Eval: 3d obj Acc@1  :" in line:
                obj_acc_1 = float(line.split(":")[-1].strip())
            if "Eval: 3d rel Acc@1  :" in line:
                rel_acc_1 = float(line.split(":")[-1].strip())
            if "Eval: 3d triplet Acc@50 :" in line:
                triplet_acc_50 = float(line.split(":")[-1].strip())
            if structured and "Total Parameters:" in line:
                total_params = int(line.split(":")[-1].strip().replace(',', ''))
            if not structured and "Total Parameters after Unstructured pruning:" in line:
                total_params = int(line.split(":")[-1].strip().replace(',', ''))
    return obj_acc_1, rel_acc_1, triplet_acc_50, total_params

In [3]:
def process_folders(base_path, structured=True):
    results = []
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            file_path = os.path.join(folder_path, folder_path, 'result.txt')
            if os.path.isfile(file_path):
                obj_acc_1, rel_acc_1, triplet_acc_50, total_params = extract_values(file_path, structured)
                if obj_acc_1 is not None and rel_acc_1 is not None and triplet_acc_50 is not None and total_params is not None:
                    if 'baseline' in folder_name:
                        ratio = 0
                    else:
                        ratio = re.search(r'st(\d+)', folder_name)
                        if ratio:
                            ratio = int(ratio.group(1)) / 100
                    results.append((ratio, obj_acc_1, rel_acc_1, triplet_acc_50, total_params))
    return results

In [4]:
def plot_results(title, structured_results, unstructured_results, additional_structured_results, additional_unstructured_results):
    structured_results.sort()
    unstructured_results.sort()
    additional_structured_results.sort()
    additional_unstructured_results.sort()

    fig, axs = plt.subplots(3, 1, figsize=(10, 18))
    
    # Baseline 값 찾기 및 분리
    baseline_structured = [result for result in structured_results if result[0] == 0]
    baseline_unstructured = [result for result in unstructured_results if result[0] == 0]

    # Baseline을 제외한 나머지 structured 결과
    structured_results = [result for result in structured_results if result[0] != 0]
    unstructured_results = [result for result in unstructured_results if result[0] != 0]

    if structured_results:
        struct_ratios, struct_obj_accs, struct_rel_accs, struct_triplet_accs, _ = zip(*structured_results)
    else:
        struct_ratios, struct_obj_accs, struct_rel_accs, struct_triplet_accs = [], [], [], []

    if unstructured_results:
        unstruct_ratios, unstruct_obj_accs, unstruct_rel_accs, unstruct_triplet_accs, _ = zip(*unstructured_results)
    else:
        unstruct_ratios, unstruct_obj_accs, unstruct_rel_accs, unstruct_triplet_accs = [], [], [], []

    if additional_structured_results:
        add_struct_ratios, add_struct_obj_accs, add_struct_rel_accs, add_struct_triplet_accs = zip(*additional_structured_results)
    else:
        add_struct_ratios, add_struct_obj_accs, add_struct_rel_accs, add_struct_triplet_accs = [], [], [], []

    if additional_unstructured_results:
        add_unstruct_ratios, add_unstruct_obj_accs, add_unstruct_rel_accs, add_unstruct_triplet_accs = zip(*additional_unstructured_results)
    else:
        add_unstruct_ratios, add_unstruct_obj_accs, add_unstruct_rel_accs, add_unstruct_triplet_accs = [], [], [], []

    # Obj Acc@1 그리기
    axs[0].set_title('3d obj Acc@1')
    axs[0].set_xlabel('Pruning Ratio')
    axs[0].set_ylabel('Acc@1')
    axs[0].grid(True)
    axs[0].set_xticks([i/100 for i in range(0, 76, 5)])

    if baseline_structured:
        _, baseline_acc, _, _, _ = baseline_structured[0]
        axs[0].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[0].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if baseline_unstructured:
        _, baseline_acc, _, _, _ = baseline_unstructured[0]
        axs[0].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[0].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if structured_results:
        axs[0].plot(struct_ratios, struct_obj_accs, marker='o', linestyle='-', label='Structured Pruning', color='#6C946F')

    if additional_structured_results:
        axs[0].plot(add_struct_ratios, add_struct_obj_accs, marker='*', linestyle='--', label='Structured Pruning + Fine-tuning', color='#6C946F')
        for ratio, acc in zip(add_struct_ratios, add_struct_obj_accs):
            axs[0].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    if unstructured_results:
        axs[0].plot(unstruct_ratios, unstruct_obj_accs, marker='o', linestyle='-', label='Unstructured Pruning', color='#DC0083')

    if additional_unstructured_results:
        axs[0].plot(add_unstruct_ratios, add_unstruct_obj_accs, marker='*', linestyle='--', label='Unstructured Pruning + Fine-tuning', color='#DC0083')
        for ratio, acc in zip(add_unstruct_ratios, add_unstruct_obj_accs):
            axs[0].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    axs[0].legend()

    # Rel Acc@1 그리기
    axs[1].set_title('3d rel Acc@1')
    axs[1].set_xlabel('Pruning Ratio')
    axs[1].set_ylabel('Acc@1')
    axs[1].grid(True)
    axs[1].set_xticks([i/100 for i in range(0, 76, 5)])

    if baseline_structured:
        _, _, baseline_acc, _, _ = baseline_structured[0]
        axs[1].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[1].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if baseline_unstructured:
        _, _, baseline_acc, _, _ = baseline_unstructured[0]
        axs[1].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[1].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if structured_results:
        axs[1].plot(struct_ratios, struct_rel_accs, marker='o', linestyle='-', label='Structured Pruning', color='#6C946F')

    if additional_structured_results:
        axs[1].plot(add_struct_ratios, add_struct_rel_accs, marker='*', linestyle='--', label='Structured Pruning + Fine-tuning', color='#6C946F')
        for ratio, acc in zip(add_struct_ratios, add_struct_rel_accs):
            axs[1].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    if unstructured_results:
        axs[1].plot(unstruct_ratios, unstruct_rel_accs, marker='o', linestyle='-', label='Unstructured Pruning', color='#DC0083')

    if additional_unstructured_results:
        axs[1].plot(add_unstruct_ratios, add_unstruct_rel_accs, marker='*', linestyle='--', label='Unstructured Pruning + Fine-tuning', color='#DC0083')
        for ratio, acc in zip(add_unstruct_ratios, add_unstruct_rel_accs):
            axs[1].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    axs[1].legend()

    # Triplet Acc@50 그리기
    axs[2].set_title('3d triplet Acc@50')
    axs[2].set_xlabel('Pruning Ratio')
    axs[2].set_ylabel('Acc@50')
    axs[2].grid(True)
    axs[2].set_xticks([i/100 for i in range(0, 76, 5)])

    if baseline_structured:
        _, _, _, baseline_acc, _ = baseline_structured[0]
        axs[2].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[2].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if baseline_unstructured:
        _, _, _, baseline_acc, _ = baseline_unstructured[0]
        axs[2].scatter(0, baseline_acc, color='#FF0000', label='Baseline', zorder=5)
        axs[2].text(0, baseline_acc, f'{baseline_acc:.2f}', fontsize=9, verticalalignment='bottom')

    if structured_results:
        axs[2].plot(struct_ratios, struct_triplet_accs, marker='o', linestyle='-', label='Structured Pruning', color='#6C946F')

    if additional_structured_results:
        axs[2].plot(add_struct_ratios, add_struct_triplet_accs, marker='*', linestyle='--', label='Structured Pruning + Fine-tuning', color='#6C946F')
        for ratio, acc in zip(add_struct_ratios, add_struct_triplet_accs):
            axs[2].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    if unstructured_results:
        axs[2].plot(unstruct_ratios, unstruct_triplet_accs, marker='o', linestyle='-', label='Unstructured Pruning', color='#DC0083')

    if additional_unstructured_results:
        axs[2].plot(add_unstruct_ratios, add_unstruct_triplet_accs, marker='*', linestyle='--', label='Unstructured Pruning + Fine-tuning', color='#DC0083')
        for ratio, acc in zip(add_unstruct_ratios, add_unstruct_triplet_accs):
            axs[2].text(ratio, acc, f'{acc:.2f}', fontsize=9, verticalalignment='bottom')

    axs[2].legend()

    plt.tight_layout()
    plt.suptitle(title, y=1.02, fontsize=16)
    plt.show()

In [10]:
title = 'SGPN Structured, Unstructured Pruning Performance'
base_path_structured = '/home/knuvki/Desktop/song/VLSAT_pruning/config/results/sgpn/eval_sgpn_st'  # 여기에 실제 구조화된 폴더 경로를 넣으세요
base_path_unstructured = '/home/knuvki/Desktop/song/VLSAT_pruning/config/results/sgpn/eval_sgpn_unst'  # 여기에 실제 비구조화된 폴더 경로를 넣으세요

structured_results = process_folders(base_path_structured, structured=True)
unstructured_results = process_folders(base_path_unstructured, structured=False)    
# 추가 학습 후 결과 직접 입력 (구조화된 및 비구조화된)
additional_structured_results = [
    (0.25, 52.37, 82.93, 87.32),  # (Pruning Ratio, 3d triplet Acc@50)
]
additional_unstructured_results = [
    (0.75, 52.3, 82.41, 87.06)
]

for ratio, obj_acc_1, rel_acc_1, triplet_acc_50, _ in structured_results:
    print(f"Structured - Pruning Ratio: {ratio}, Eval: 3d obj Acc@1: {obj_acc_1}, Eval: 3d rel Acc@1: {rel_acc_1}, Eval: 3d triplet Acc@50: {triplet_acc_50}")

for ratio, obj_acc_1, rel_acc_1, triplet_acc_50, _ in unstructured_results:
    print(f"Unstructured - Pruning Ratio: {ratio}, Eval: 3d obj Acc@1: {obj_acc_1}, Eval: 3d rel Acc@1: {rel_acc_1}, Eval: 3d triplet Acc@50: {triplet_acc_50}")

for ratio, obj_acc_1, rel_acc_1, triplet_acc_50 in additional_structured_results:
    print(f"Additional Structured Training - Pruning Ratio: {ratio}, Eval: 3d obj Acc@1: {obj_acc_1}, Eval: 3d rel Acc@1: {rel_acc_1}, Eval: 3d triplet Acc@50: {triplet_acc_50}")

for ratio, obj_acc_1, rel_acc_1, triplet_acc_50 in additional_unstructured_results:
    print(f"Additional Unstructured Training - Pruning Ratio: {ratio}, Eval: 3d obj Acc@1: {obj_acc_1}, Eval: 3d rel Acc@1: {rel_acc_1}, Eval: 3d triplet Acc@50: {triplet_acc_50}")

plot_results(title, structured_results, unstructured_results, additional_structured_results, additional_unstructured_results)

Structured - Pruning Ratio: 0.05, Eval: 3d obj Acc@1: 54.73129610115912, Eval: 3d rel Acc@1: 87.96043825293738, Eval: 3d triplet Acc@50: 89.36096375985325
Structured - Pruning Ratio: 0.35, Eval: 3d obj Acc@1: 54.035827186512115, Eval: 3d rel Acc@1: 81.00986564870358, Eval: 3d triplet Acc@50: 88.81810520053543
Structured - Pruning Ratio: 0.25, Eval: 3d obj Acc@1: 54.689146469968385, Eval: 3d rel Acc@1: 84.20504684943731, Eval: 3d triplet Acc@50: 89.20232016260968
Structured - Pruning Ratio: 0.45, Eval: 3d obj Acc@1: 52.075869336143306, Eval: 3d rel Acc@1: 72.7951018789351, Eval: 3d triplet Acc@50: 87.93812899707501
Structured - Pruning Ratio: 0.1, Eval: 3d obj Acc@1: 54.81559536354057, Eval: 3d rel Acc@1: 87.33825789499777, Eval: 3d triplet Acc@50: 89.43284913985424
Structured - Pruning Ratio: 0.6, Eval: 3d obj Acc@1: 42.40252897787144, Eval: 3d rel Acc@1: 66.37499380298448, Eval: 3d triplet Acc@50: 81.22304298249963
Structured - Pruning Ratio: 0.4, Eval: 3d obj Acc@1: 53.02423603793467

ValueError: not enough values to unpack (expected 4, got 2)